## Example: Reading point location data 

This example illustrates the how to read point location data using the HydroMT [DataCatalog](../_generated/hydromt.data_catalog.DataCatalog.rst) with the `vector` and `netcdf` or `zarr` drivers.

In [ ]:
# import hydromt and setup logging
import hydromt
from hydromt.log import setuplog

logger = setuplog("read point location data", log_level=10)

In [ ]:
# Download artifacts for the Piave basin to `~/.hydromt_data/`.
data_catalog = hydromt.DataCatalog(logger=logger)
data_catalog.from_predefined_catalogs('artifact_data')

## Netcdf or zarr driver

To read point location data and parse it into a [xarray Dataset or DataArray](https://docs.xarray.dev/en/stable/user-guide/data-structures.html) we use the [open_mfdataset()](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset) or the [open_zarr()](https://docs.xarray.dev/en/stable/generated/xarray.open_zarr.html#xarray.open_zarr) method. All `kwargs` in the data catalog yaml file will be passed to to these methods. 

As an example we will use the [GTSM data](hhttps://cds.climate.copernicus.eu/cdsapp#!/dataset/10.24381/cds.8c59054f?tab=overview) dataset which is stored in Netcdf format. 

In [ ]:
# inspect data source entry in data catalog yaml file
data_catalog["gtsmv3_eu_era5"]

We can load any point location data using [DataCatalog.get_geodataset()](../_generated/hydromt.data_adapter.DataCatalog.get_geodataset.rst). Note that if we don't provide any arguments it returns the full dataset with all data variables and for the full spatial domain.  The result is per default returned as DataArray as the dataset consists of a single variable. To return a dataset  use the `single_var_as_array=False` argument. Only the data coordinates and the time are actually read, the data variables are still lazy [Dask arrays](https://docs.dask.org/en/stable/array.html).

In [ ]:
ds = data_catalog.get_geodataset("gtsmv3_eu_era5", single_var_as_array=False)
ds

The data can be visualized with the [.plot()](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.plot.html) xarray method. We show the evolution of the water level over time for a specific point location (station). 

In [ ]:
ds.sel(stations=2791)["waterlevel"].plot()

We can request a (spatial) subset data by providing additional `variables` and `bbox` / `geom` arguments. Note that these return less stations and less variables. In this example only spatial arguments are applied as only a single variable is available. The variables argument is especially useful if each variable of the dataset is saved in a separate file and the `{variable}` key is used in the path argument of the data source to limit which files are actually read. If a single variable is requested a DataArray instead of a Dataset is returned unless the `single_var_as_array` argument is set to False (True by default).

In [ ]:
bbox = [12.50, 45.20, 12.80, 45.40]
ds_bbox = data_catalog.get_geodataset("gtsmv3_eu_era5", bbox=bbox)
ds_bbox

With a Geodataset, you can also directly access the point locations using its [to_gdf](https://deltares.github.io/hydromt/latest/_generated/hydromt.DataArray.vector.to_gdf.html#hydromt.DataArray.vector.to_gdf) method:

In [ ]:
ds_bbox.vector.to_gdf()

In [ ]:
# Plot
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs

proj = ccrs.PlateCarree()

fig = plt.figure(figsize=(3, 4))
ax = plt.subplot(projection=proj)

bbox = gpd.GeoDataFrame(geometry=[box(12.50, 45.20, 12.80, 45.40)], crs=4326)

ax.add_image(cimgt.QuadtreeTiles(), 12)
# Plot the points
ds.vector.to_gdf().plot(ax=ax, markersize=30, c="blue", zorder=2)
ds_bbox.vector.to_gdf().plot(ax=ax, markersize=40, c="red", zorder=2)
# Plot the bounding box
bbox.boundary.plot(ax=ax, color="red", linewidth=0.8)

## Vector driver

To read vector data and parse it into a [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) object we use the [open_geodataset](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_geodataset.html#hydromt.data_catalog.DataCatalog.get_geodataset) method. Combined point locations (e.g. CSV or GeoJSON) data as well as text delimited time series (e.g. CSV) data are supported as file formats (see [DataCatalog documentation](https://deltares.github.io/hydromt/latest/user_guide/data_types.html#csv-point-time-series-data)). Both formats must contain an index and a crs has to be indicated in the .yml file. For demonstration we use dummy example data from the *examples/data* folder. 

First load the data catalog of the corresponding example data *geodataset_catalog.yml*:

In [ ]:
import os
geodata_catalog = hydromt.DataCatalog('data/geodataset_catalog.yml', logger=logger)
geodata_catalog['waterlevels_txt']

We see here that our locations are defined in a *data/stations.csv* file and a *data/stations_data.csv*. Let's check the content of these files before loading them with the `get_geodataset` method:

In [ ]:
import pandas as pd
from pprint import pprint

print("Stations locations:")
df_stations = pd.read_csv('data/stations.csv')
pprint(df_stations)

print("Stations data:")
df_stations_data = pd.read_csv('data/stations_data.csv')
pprint(df_stations_data.head())

In [ ]:
ds = geodata_catalog.get_geodataset("waterlevels_txt", single_var_as_array=False)
ds

In [ ]:
# Small plot for station 1001
ds.sel(stations=1001)['waterlevel'].plot()